In [1]:
import json
import string
from openpyxl import workbook, load_workbook
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# create a function to generate the percentage ranges

def make_ranges_for_std_curve(n):
    '''
    Takes n as an argument, the number of times 100% is halved.
    Returns a list of percentage ranges for the standard curve.
    The ranges are in the form of a list of percentages, 
    starting from 100% and going down to 0% in steps of 50%.

    Returns: a list of percentages.
    '''
    perc = 100
    vwf_ref_prec = []
    i = n
    while i >= 0 :
        vwf_ref_prec.append(perc)
        perc = perc / 2             # halving the percentage
        i-=1                        # decrementing the counter
        
    vwf_ref_prec.append(0)          # add 0% to the list
    return vwf_ref_prec

make_ranges_for_std_curve(6)

[100, 50.0, 25.0, 12.5, 6.25, 3.125, 1.5625, 0]

In [3]:
def get_template_from_config():
    with open('config/config.json', 'r') as file:
        config = json.load(file)
    
    return config
    

In [4]:
get_template_from_config()

{'analysis_name_pos': {'col': 1, 'row': 1},
 'serial_name_pos': {'col': 1, 'row': 2},
 'plates': [{'name': 'Plate1',
   'pos': {'col': 1, 'row': 5},
   'assay_name_pos': {'col': 1, 'row': 3}},
  {'name': 'Plate2',
   'pos': {'col': 1, 'row': 26},
   'assay_name_pos': {'col': 1, 'row': 24}}]}

In [5]:
def read_data_to_dataframe(input_file):
    '''
    Takes the path to an input file
    returns a dataframe
    '''
    data = pd.DataFrame()
    config = get_template_from_config()

    excel_to_csv = pd.read_excel(input_file, header=None)
    #excel_to_csv.to_csv("work_dir/data.csv", index=False)

    analysis_name = excel_to_csv.iloc[config['analysis_name_pos']['row']-1, 
                                    config['analysis_name_pos']['col']-1
                                    ]
    serial_name = excel_to_csv.iloc[config['serial_name_pos']['row']-1, 
                                    config['serial_name_pos']['col']-1
                                    ]
    data = pd.DataFrame()

    for plate in config["plates"]:
        row = plate['pos']['row']
        col = plate['pos']['col']
        
        tmpDf = pd.DataFrame(excel_to_csv.iloc[row-1:row+7, col:col+12])
        tmpDf['analysis_name'] = analysis_name
        tmpDf['serial_name'] = serial_name
        
        target_name = excel_to_csv.iloc[
            plate["assay_name_pos"]['row']-1, 
            plate["assay_name_pos"]['col']-1
            ]
        tmpDf['assay'] = target_name

        data = pd.concat([data, tmpDf])
 
    return data

data = read_data_to_dataframe("../../Raw_data_from_Rethabile/VWF AG & CB 1 JUNE 2025.xlsx")

data

,1,2,3,4,5,6,7,8,9,10,11,12,analysis_name,serial_name,assay
4,0.554,0.552,0.491,0.483,0.471,0.576,0.505,0.493,0.331,0.312,0.011,0.010,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
5,0.447,0.444,0.328,0.344,0.338,0.355,0.361,0.341,0.184,0.186,0.002,0.003,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
6,0.325,0.327,0.364,0.364,0.214,0.245,0.348,0.359,0.305,0.356,0.004,0.004,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
7,0.217,0.209,0.235,0.216,0.138,0.141,0.210,0.209,0.217,0.212,0.003,0.004,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
8,0.145,0.128,0.360,0.333,0.220,0.216,0.142,0.138,0.155,0.176,0.003,0.005,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
9,0.073,0.072,0.219,0.218,0.144,0.131,0.072,0.069,0.087,0.089,0.003,0.004,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
10,0.050,0.040,0.316,0.323,0.391,0.364,0.114,0.092,0.003,0.004,0.005,0.003,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
11,0.014,0.013,0.272,0.263,0.276,0.288,0.049,0.050,0.003,0.003,0.004,0.012,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:AG
25,1.449,1.535,1.070,1.116,1.101,1.138,1.185,1.339,0.718,0.830,0.021,0.021,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:CB 01 JUNE 2025
26,1.522,1.391,0.735,0.838,0.805,0.833,0.931,0.879,0.419,0.478,0.019,0.018,Date : 02 JUNE 2025,Synergy HT Serial#204811,VWF:CB 01 JUNE 2025


In [6]:
def logistic_4_param(x, a, b, c, d):
    return d + ( (a - d) / (1 + (x / c)**b) )


def compute_4PL(y, a, b, c, d):
    """
    x = c * ((a - d) / (y - d) - 1) ** (1 / b)
    """
    if y == d:
        raise ValueError("Division by zero: y must not be equal to d.")
    inner = (a - d) / (y - d) - 1
    if inner < 0 and b % 2 == 0:
        raise ValueError("Cannot compute even root of a negative number.")
    x = c * (inner) ** (1 / b)
    return x

In [7]:
def compute_4PL_params():
    unique_assays = data['assay'].unique()

    p0 = [0, 1, 25, 1] # this may need work for the inititial predictions

    param_results_dict = {}

    for assay in unique_assays:
        print(assay)
        # create a datafram to make things easier, these value are then passed to a dictionary
        standard_curve_data = data.query("assay == @assay").iloc[:,0:2]
        standard_curve_data['mean'] = standard_curve_data.mean(axis=1)
        standard_curve_data['diffs_of_values'] = abs(standard_curve_data.iloc[:,0] - standard_curve_data.iloc[:,1]) 
        standard_curve_data['perc'] = make_ranges_for_std_curve(6)

        callibrators = standard_curve_data["mean"].values
        perc = standard_curve_data["perc"].values
        diffs_of_values = standard_curve_data['diffs_of_values'].values
        
        popt, pcov = curve_fit(logistic_4_param, perc, callibrators, p0=p0)
        
        # Extract the fitted parameters
        a_fit, b_fit, c_fit, d_fit = popt

        # Generate the fitted curve
        x_fit = np.linspace(min(perc), max(perc)*2, 10000)
        y_fit = logistic_4_param(x_fit, a_fit, b_fit, c_fit, d_fit)

        param_results_dict[assay] = {"params": popt, 
                            "perc": perc, 
                            "callibrators": callibrators,
                            "diffs_of_values": diffs_of_values,
                            "x_fit": x_fit, 
                            "y_fit": y_fit}
        
    return param_results_dict
    

In [8]:
param_results_dict = compute_4PL_params()
param_results_dict

VWF:AG 
VWF:CB 01 JUNE 2025


/tmp/ipykernel_1701225/1338541779.py:2: RuntimeWarning: invalid value encountered in power
  return d + ( (a - d) / (1 + (x / c)**b) )


{'VWF:AG ': {'params': array([1.17434199e-02, 9.84316756e-01, 3.23366459e+01, 7.30174910e-01]),
  'perc': array([100.    ,  50.    ,  25.    ,  12.5   ,   6.25  ,   3.125 ,
           1.5625,   0.    ]),
  'callibrators': array([0.553 , 0.4455, 0.326 , 0.213 , 0.1365, 0.0725, 0.045 , 0.0135]),
  'diffs_of_values': array([0.002, 0.003, 0.002, 0.008, 0.017, 0.001, 0.01 , 0.001]),
  'x_fit': array([0.00000000e+00, 2.00020002e-02, 4.00040004e-02, ...,
         1.99959996e+02, 1.99979998e+02, 2.00000000e+02], shape=(10000,)),
  'y_fit': array([0.01174342, 0.01224206, 0.01272924, ..., 0.62768103, 0.62768969,
         0.62769834], shape=(10000,))},
 'VWF:CB 01 JUNE 2025': {'params': array([ 0.06179516,  1.14811061, 11.79821397,  1.66107843]),
  'perc': array([100.    ,  50.    ,  25.    ,  12.5   ,   6.25  ,   3.125 ,
           1.5625,   0.    ]),
  'callibrators': array([1.492 , 1.4565, 1.2145, 0.8325, 0.581 , 0.3645, 0.228 , 0.041 ]),
  'diffs_of_values': array([0.086, 0.131, 0.053, 0.029,

In [19]:
for key in param_results_dict.keys():
    fig = go.Figure()
    # Add the points for the percentage calibrator and Absorbance
    fig.add_trace(
        go.Scatter(
            x=param_results_dict[key]["perc"],
            y=param_results_dict[key]["callibrators"],
            error_y=dict(
                type='data', 
                array=param_results_dict[key]["diffs_of_values"],
                visible=True),
            mode='markers',
            marker=dict(
            size=10,        
            color='teal',   
            symbol='circle' 
            )
            )
        ),
    # Add a line plot for the fitted model
    fig.add_trace(
        go.Scatter(
            x=param_results_dict[key]["x_fit"],
            y=param_results_dict[key]["y_fit"],
            mode='lines'
        )
        
    )
    # Update the figure size, margin and title
    fig.update_layout(
        autosize=False,
        width=600,
        height=400,
        margin=dict(
            l=50,
            r=50,
            b=10,
            t=40,
            pad=4
            ),
        title=dict(text=key, font=dict(size=20)),
        xaxis=dict(title=dict(text="Percentage")),
        yaxis=dict(title=dict(text="Absorbance"))
        
        )
    fig.show()